# Wczytanie danych
W trakcie zajęć będziemy pracować nad podstawowym zbiorem danych dotyczących teksów z platformy [Kaggle](https://www.kaggle.com/competitions/nlp-getting-started/overview)

W pierwszej kolejności wczytajmy dane za pomocą pakietu *pandas*. Interesuje nas plik *train.csv*

Organizujemy nazwę ścieżki pliku:

In [ ]:
import os
filePath = os.getcwd() + "/train.csv"
filePath

W tym bloku wczytujemy dane:

In [ ]:
import pandas as pd

df = pd.read_csv(filePath)
df.head(5)

# Preprocesowanie tekstu
W pierwszej kolejności przygotowujemy tekst do pracy, tak jak robiliśmy to na poprzednich zajęciach - w pierwszej kolejnosci usuńmy cyfry, wielkie litery etc.

In [ ]:
import nltk
import re

def linkRemoval(text):
    text = re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", text)
    text = re.sub(r'\d+', '', text)
    return text

df['tokenized_texts'] = df['text'].str.lower()
df['tokenized_texts'] = df['tokenized_texts'].apply(linkRemoval)

df.head(5)

Następnie rozdzielamy na wyrazy:

In [ ]:
tokenizer = nltk.RegexpTokenizer(r"\w+")
df['tokenized_list'] = df['tokenized_texts'].apply(tokenizer.tokenize)
df.head(5)

I usuwamy puste słowa:

In [ ]:
nltk.download('stopwords')
stop = nltk.corpus.stopwords.words('english')
df['tokenized_list'] = df['tokenized_list'].apply(lambda x: [word for word in x if word not in (stop)])
df.head(5)


Dokonujemy stemmingu i lematyzacji:

In [ ]:
nltk.download('wordnet')

def stemmer_text(word_list):
    stemmer = nltk.stem.porter.PorterStemmer()
    return [stemmer.stem(w) for w in word_list]

def lemmatize_text(word_list):
    lemmatizer = nltk.stem.WordNetLemmatizer()
    return [lemmatizer.lemmatize(w) for w in word_list]

df['tokenized_list'] = df['tokenized_list'].apply(stemmer_text)
df['tokenized_list'] = df['tokenized_list'].apply(lemmatize_text)
df.head(5)

In [ ]:
df['final_string'] = df['tokenized_list'].apply(lambda x: ' '.join([str(i) for i in x]))

# Chmura słów
Dla zwizualizowania sobie danych stworzymy prostą chmurę słów.

In [ ]:
def printWordcloud(df_column):
    from wordcloud import WordCloud
    import matplotlib.pyplot as plt
    list_Comments = df_column.to_list()
    string_comments = ' '.join(list_Comments)

    wordcloud = WordCloud(background_color="white",
                          width=800, height=400).generate(string_comments)

    plt.figure()
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.show()

printWordcloud(df.final_string)

# Modelowanie tematów - algorytm BERT
Istnieje kilka algorytmów, które pozwalają określić podobieństwo różnych wypowiedzi - teraz zobaczymy algorytm bazujący na modelu językowy Google - BERT.

Paczka BERTopic bazować będzie na:
1.   Redukcji wymiaru danych - [UMAP](https://umap-learn.readthedocs.io/en/latest/index.html)
2.   Wydzielanie klastrów - [HDBScan](https://hdbscan.readthedocs.io/en/latest/index.html)
3.   Sieciach neuronowych - tutaj będą to implementacje bibliotek [Keras](https://keras.io/) i [Tensorflow](https://www.tensorflow.org/)



In [ ]:
pip install bertopic

Pierwszym krokiem jest dopasowanie sieci neuronowej do danych - warstwy zawierają poprzednio wskazane algorytmy:

In [ ]:
from bertopic import BERTopic
model = BERTopic()
topics, probs = model.fit_transform(df.final_string.tolist())

Możemy przypisać tematy do naszych tweetów:

In [ ]:
df['topic'] = topics
df.head(10)

W pierwszej kolejności możemy zobaczyć jak algorytm dobrał teksty w wątki.
1.   Pod liczbą -1 ukrywają się posty jednoznacznie niesklasyfikowane jako powtarzalne wzroce.
2.   Każde kolejne grupy zawierać będa słowa kluczowe oraz prawdopodobieństwo pojawienia się słowa w tekście.


In [ ]:
df_results =  model.get_topic_info()
df_results.head(10)

#Same częstotliwości
#model.get_topic_freq()

Sieć neuronowa określa również prawdopodobieństwo pojawienia się danego słowa w tekście o określonym temacie:

In [ ]:
results = model.get_topics()
model.get_topic(0)

Mamy też trochę szerszy interfejs do przedstawiania dokumentów:

In [ ]:
df_classification =  model.get_document_info(df.final_string)
df_classification.head(10)

Kilka gotowych wykresów:

In [ ]:
model.visualize_topics()

In [ ]:
model.visualize_barchart()

In [ ]:
model.visualize_heatmap()

Więcej informacji ma [Github.io twórcy](https://maartengr.github.io/BERTopic/index.*html*).

Szybkie ograniczenie ramki do tekstów z danego tematu dostępne w pakiecie Pandas:

In [ ]:
df.query("topic == 2")

# Analiza Sentymentu - Algorytm VADER
Kolejnym częstym zastosowaniem pojawiającym się w analizach tekstów jest określenie czy są one pozytywne czy negatywne. Bardziej złożone algorytmy potrafią wskazać też emocje takie jak strach, złość czy radość.

Przejrzymy najprostrzy algorytm VADER - zobaczmy jego funkcjonowanie dla pierwszego tekstu. Trzy linijki kodu poniżej generują obiekt, który buduje analizy sentymentu.

In [ ]:
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentAnalyzer
sia = nltk.sentiment.SentimentIntensityAnalyzer()

To będzie nasz tekst na którym pracujemy:

In [ ]:
tekst = df.text.iloc[1]
tekst

Co generuje algorytm?

Mamy słownik, tudzież ramkę JSON. Pokazuje procentowo jaka część zdania ma słowa pozytywne, negatywne i neutralne. Na koniec indeks zbiorczy.

In [ ]:
sia.polarity_scores(tekst)

Zaaplikujmy to do całej ramki danych. Wygenerujemy funkcję, którą przekażemy do każdego tekstu w ramce.

In [ ]:
def sentiment(word_list):
    sia = nltk.sentiment.SentimentIntensityAnalyzer()
    tempString = ' '.join(word_list)
    return sia.polarity_scores(tempString)

df['sentiment'] = df['tokenized_list'].apply(sentiment)
df.head(5)

Możemy rozłożyć ramkę na kolejne kolumny - robi się to tak:

In [ ]:
df2 = pd.json_normalize(df.sentiment)
df = pd.concat([df, df2], axis=1)

Stwórzmy własny indeks dyfuzyjny:

In [ ]:
df["VADER_index"] = (df.pos - df.neg)/(df.pos + df.neg)
df.head(10)

# Ngramy i tagowanie częsci zdania
Na koniec jeszcze dwa zastosowania:
1.   Możemy szukać określonych zbitków słów następujących po sobie - to są tzw. N-gramy. Dwa słowa to Bigram, Trzy słowa trigram etc.
2.   Możemy też otagować każde ze słów - czy to czasownik, przymiotnik etc.

In [ ]:
speech_list = df['final_string'].to_list()
speech_string = ' '.join(speech_list)
words = speech_string.split()

In [ ]:
n = 3
ngrams_obj = nltk.ngrams(words, n)

fdist = nltk.FreqDist(ngrams_obj)

Stosunkowo prosto możemy utrworzyć sobie ramkę z najczęściej powtarzającymi się słowami:

In [ ]:
frame = pd.DataFrame(fdist.items() , columns=['Words','Frequency'])
frame = frame[frame['Frequency'] >= 10]
frame.sort_values('Frequency', ascending=False).head(10)

Tagowanie jest stosunkowo proste:

In [ ]:
nltk.download('averaged_perceptron_tagger')

def tagowanie(tekst):
  tokenizer = nltk.RegexpTokenizer(r"\w+")
  tokenized_list = tokenizer.tokenize(tekst)
  return nltk.pos_tag(tokenized_list)

df["POS"] = df.text.apply(tagowanie)

Przyjrzymy się wynikom - lista tego co oznaczają kolejne tagi dostępna jest [tutaj](https://www.learntek.org/blog/categorizing-pos-tagging-nltk-python/).

In [ ]:
df[["text","POS"]].head(10)